# PYNQ Pattern Generator LED string example

* First version: 16 March 2021

* Updated: 9 April 2021 (Hackathon release)

Author: CMC

## Requirements

PYNQ v2.6 with custom changes to pattern_generator.py. Updated pattern_generator.py should be included with this notebook and must be copied to the board to overwrite the existing pattern_generator.py before running. 

## Introduction

* This notebook shows how to use the PYNQ logictools *Pattern Generator* to  control multi-color LED strings. 

The notebook covers:

* Background information on LED strings
* Information about timing specification and timing for the most popular type of LED strings
* Instructions to connect a Grove LED string to the PYNQ-Z2 board
* Python code to use the logictools Pattern Generator to generate patterns to control the LEDs

## Table of Contents

* [Background](#Background)
* [Grove LED stick](#Grove-LED-stick)
* [Board set up](#Board-set-up)

## Background 

Low-cost RGB LED *strings* are available in many formats and can be used for a variety of interesting projects. The LEDs are available in different formats. You can get individual LEDs, strings, rings, and panels or arrays. 

https://www.adafruit.com/category/168

Individual LEDs can be wired together, and you can use a string of LEDs which are flexible, so you can share them to a surface or object to make interesting displays. 

Neopixels is a popular brand name for these LED strings. "WS2812" or "WS2813" is the technical name for the  controller or protocol used to control these LEDs.  

## Grove LED stick

The Grove RGB LED stick has 10 WS2813 LEDs: 
https://wiki.seeedstudio.com/Grove-RGB_LED_Stick-10-WS2813_Mini/

We will use a PYNQ enabled board to build a controller for the LED stick. 

Ref: https://en.wikipedia.org/wiki/LED_strip_light

<div class="alert alert-box alert-info">
   
Note: WS2813 is a newer version of the WS2813. This notebook can be used with either type of LED.</div>

### Controller Protocol

Each LED has a separate Red, Green and Blue part. These three parts can be turned on or off independently to create different colors. Pixels in most TVs, computer monitors or phone screens work in this way. The LEDs are bigger than a "pixel" and can be much brighter, allowing you to make different types of displays. 

The LEDs work by writing a value to each color component. Each color value is 8-bits (or eight 1's or 0's). The value for each color can be from 0 (all zeros, or fully "off") to 255 (all ones or fully "on"). Combining all three 8-bit values gives us a 24-bit value for one LED. This allows us to generate over 16 Million colors for each LED. 

If you only have one LED, you only need to write one 24-bit value - 24 ones or zeros. If you have a string of LEDs, you can send one 24-bit LED value after another. The stream of data will be passed down the chain and a value will be assigned to each LED. 

The LED controller uses a serial protocol to send values to the LEDs. This means that the controller only uses 1 wire to send values to the LEDs. The controller can only write a ONE or a ZERO on this wire to send information. 
The serial protocol also means that you can connect long chains of LEDs together. In theory, there is no upper limit on the number of LEDs you can connect.

### Timing

The color values we want to send will be transmitted by setting the value of the signal we write to 1 for some amount of time, and then 0 for another specific amount of time. By varying the time that the signal is 1 and 0 will determine if a value of 1 or 0 is sent. The LED controller will decode the signal and use the value we send to turn on the LED. 

For the WS2812, to send a ZERO, we will set the signal high for at least 200 nanoseconds and not more than 500 nanoseconds, and low for at least 450 nanoseconds (and not more than 5500 nanoseconds).

To send a ONE we will set the signal high for at least 550 nanoseconds and low for at least 450 nanoseconds. 


These are the timings we need:


|<td colspan=1>Simplified timing WS2812   |
| --- | ----- | --- | --- | --- | --- | ---
|Symbol	| Parameter	            | Min	| Typical	| Max	| Units | PG pulses @ 10MHz | PG pulses @ 5MHz | 
|T0H	    | 0 code high voltage time	| 200	| 350	    | 500	| ns | 3 | 1
|T1H	    | 1 code high voltage time	| 550	| 700	    | 5,500	| ns | 6 | 3 
|T0L	data | low voltage time	    | 450	| 600	    | 5,000	| ns | 5 | 3 (this may not meet spec)
|T1L	data | low voltage time	    | 450	| 600	    | 5,000	| ns | 5 | 3 (this may not meet spec)
|TLL	latch|  low voltage time	| 6,000	| 		    |       | ns | 60 | 30

If we hold low for >6000 ns, it will rest (ready to send next pattern to LED string)
Ref: https://wp.josh.com/2014/05/13/ws2812-neopixels-are-not-so-finicky-once-you-get-to-know-them/
https://www.arrow.com/en/research-and-events/articles/protocol-for-the-ws2812b-programmable-led

|<td colspan=1>Simplified timing WS2812   |
| --- | ----- | --- | --- | --- | --- | ---
|Symbol	| Parameter	            | Min	| Max	| Units | PG pulses @ 10MHz | PG pulses @ 5MHz | 
|T0H	    | 0 code high voltage time	| 220	| 380	  | ns | 3 | ~1 (this may not meet spec)
|T1H	    | 1 code high voltage time	| 580	| 1600	  | ns | 6 | 3 
|T0L	data | low voltage time	    | 580	| 1600	    |  ns | 6 | 2 
|T1L	data | low voltage time	    | 220	| 420	    |  ns | 3 | 2 
|TLL	latch|  low voltage time	| 280	| 		    | us | 28000 | 14000

Ref: https://files.seeedstudio.com/wiki/Grove-RGB_LED_Stick-10-WS2813_Mini/res/WS2813-Mini.pdf



The PYNQ-Z2 can be used to build a controller that will send patterns at these very precise times. 

## Board set up

The following code is intended to be run on the PYNQ-Z2 board using the *logictools* overlay. Logictools includes a programmable pattern generator. This allows us to define patterns and write them out on the pins on the board. 

Any WS2812 string should work with this notebook. The pattern generator can run at different speeds. The speed or the frequency will determine the time a "sample" or value is held high or low. Based on this measurement, we can figure out how many 1s and 0s we need to generate the timed pattern for the LEDs.

### Connections:
The controller has one data wire, but all electrical systems need power and ground, so we will have 3 wires altogether. 
The WS2813 included in the PYNQ hackathon kit uses a Grove connector. This connector has 4 wires: Power, ground, data (our control wire) and the fourth wire is unconnected. The reason we have an unconnected wire is because the Grove connector is a standard connecter. It is convenient to use it so that it can easily connect to different things even though we don't use all four wires. 

* Connect the Base shield to the board. 
* Connect the Grove wire to the WS2813 LED string and to the UART port of the Base Shield. 
* Output pin D0 from the PYNQ-Z2 board is connected to the "Rx" pin of the shield, and therefore to our LED string through the Grove connector. 
* The power and ground wires will be connected via the shield. 

![](./images/IMG_2204.jpg)

### Define bit patterns for 'ones' and 'zeros'

Based on the timing values for logic tools, we will define the following values for the number of "high" and "low" pulses we need so that we can generate patterns for different LED values. 


In [ ]:
WS2812 = False 

if(WS2812 is True):
    T0H_PULSES = 3
    T1H_PULSES = 6
    T0L_PULSES = 5
    T1L_PULSES = 5
    TLL_PULSES = 60 # Not needed unless trying to send multiple sequential patterns quickly. 
else: # WS2813; 
    T0H_PULSES = 3
    T1H_PULSES = 8
    T0L_PULSES = 8 
    T1L_PULSES = 4
    TLL_PULSES = 28000 # Not needed unless trying to send multiple sequential patterns quickly. 

<div class="alert alert-box alert-info"> 
Note: The timing values inferred from the specification for the WS2813 were found to be unreliable giving incorrect color values. Adding extra samples to the values seems to make the LEDs more stable. 
</div>

Instead of '1's and '0's, *Logictools* uses patterns of 'h' and 'l' for high and low values. We will build a pattern now for a '1' and a '0' 

In [ ]:
t0h = "h" * T0H_PULSES
t1h = "h" * T1H_PULSES
t0l = "l" * T0L_PULSES
t1l = "l" * T1L_PULSES
tll = "l" * TLL_PULSES

bit0 = t0h + t0l
bit1 = t1h + t1l

Let's look at these patterns we have just created:    

In [ ]:
print("bit0: " + bit0)
print("bit1: " + bit1)

We can see bit0 has 3 'h' followed by 5 'l'. The bit1 has 5 'h' and 5 'l'. 
Go back and check the timing for these patterns matches what the LED controller requires. 

We can also check the signals visually by drawing their waveforms. 

In [ ]:
from pynq.lib.logictools.waveform import draw_wavedrom

pattern = {'signal': [{'name': 'clk', 'wave': 'hl' * 8},
                      {'name': 'bit0', 'wave': bit0},
                      {'name': 'bit1', 'wave': bit1}],
           'foot': {'tock': 1},
           'head': {'text': 'Pattern'}}

draw_wavedrom(pattern)

## Write some patterns to the LEDs. 

We can now test out some patterns on the board. First we have to load the *logictools* overlay to the board and create an instance of the pattern generator. We can use this to send values to the board. 

In [ ]:
from pynq.overlays.logictools import LogicToolsOverlay
logictools_olay = LogicToolsOverlay('logictools.bit')

* Instantiate the pattern generator 

In [ ]:
pattern_generator = logictools_olay.pattern_generator

## Setup full LED pattern
We created *bit0* and *bit1*. These are single bit values that we can reuse to build up a full 24-bit LED value. 

We could set all 24 bits to "1" e.g. 0xffffff or 0b11111111111111111111. We could do this by using bit1*24. 
However, this will turn each of the red, green, and blue LEDs to their maximum value which will be very bright. Instead, set the upper 4 bits of each color component to 0, and the lower 4 bits to 1. We will make a variable for this called 'half_on'. We will set up a red, green and blue variable with this value, and then join them together to create a 24-bit. Note the order of the colors is green, then red, then blue. 

In [ ]:
half_on = bit0*4 + bit1*4

red = half_on
green = half_on
blue = half_on

led_bit_pattern = green+red+blue

We now have a pattern to write a single pixel. We can display the string for this pattern which will be a long sequence of 'h' and 'l' values. 

In [ ]:
led_bit_pattern

Next, we will create the *Wavedrom* formatted dictionary we need to for this pattern.

In [ ]:
led_wavedrom_format = {'signal': [
    ['led',
        {'name': 'bit0', 'pin': 'D0', 'wave': led_bit_pattern}], 
    ], 
    'foot': {'tock': 1},
    'head': {'text': 'led_pattern'}}

We can now use this to set the value of a pixel. 

## Setup and run the pattern generator 

In [ ]:
pattern_generator.setup(led_wavedrom_format,
                        stimulus_group_name='led', frequency_mhz=10, initial_value=0)

In [ ]:
pattern_generator.run()

You should now see a LED turn on. 

Reset is needed before another pattern can be written by pattern generator, so we will run that next. 

In [ ]:
pattern_generator.reset()

We can now start to display other patterns. You can change the value of the green, red and blue components and rerun the cell below multiple times to see different values. 

In [ ]:
on = bit1*8
off = bit0*8

green = on+off+off
red = off+on+off
blue = off+off+on
white = on*3
black = off*3 # ALl leds "off"

led_bit_pattern = green*10 
led_wavedrom_format = {'signal': [
    ['led',
        {'name': 'bit0', 'pin': 'D0', 'wave': led_bit_pattern}], 
    ], 
    'foot': {'tock': 1},
    'head': {'text': 'led_pattern'}}
pattern_generator.setup(led_wavedrom_format,
                        stimulus_group_name='led', frequency_mhz=10, initial_value=0)

In [ ]:
pattern_generator.run()

In [ ]:
pattern_generator.reset() 

That was a basic test. Now let's do something more interesting. 

## Define some helper functions

In [ ]:
from time import sleep
def update_leds(led_bit_pattern):
    waveform = {'signal': [
        ['led',
            {'name': 'bit0', 'pin': 'D0', 'wave': led_bit_pattern}], 
        ], 
        'foot': {'tock': 1},
        'head': {'text': 'waveform'}}
    
    pattern_generator.setup(waveform,
                            stimulus_group_name='led', frequency_mhz=10, initial_value=0)
    pattern_generator.run()
    pattern_generator.reset() # Leave PG ready for next update

In [ ]:
def list_to_pattern(led_array):
    pattern = ''
    for i, _ in enumerate(led_array):
        for j in range(23, -1, -1): # walk through 24 bits
            #print(j)
            bit_test = (led_array[i] >>j) & 1 
            # Append a bit0/bit1 pattern
            if bit_test is 0:
                pattern = pattern+bit0 
            else:
                pattern = pattern+bit1 
    return pattern

## Try writing some patterns

First set up an array for the LEDs:

In [ ]:
NUMBER_OF_LEDS = 10 # Change to write to longer strings
led_array = [0] * NUMBER_OF_LEDS 

Generate some random values and update the LEDs. 

In [ ]:
import random

# Generate some random RGB values 
for i in range(NUMBER_OF_LEDS):
    led_array[i] = random.randint(0,0x0f0f0f) # The maximum random value is limited so LEDs are not too bright
led_bit_pattern = list_to_pattern(led_array)
update_leds(led_bit_pattern)

## Speed up the pattern generator

The Wavedrom format is useful for defining and displaying patterns in the notebook. However, it takes more time to process the wavedrom values, so updating the LEDs is slow. This can be speeded up by using the `update_pattern()` method which allows an array of values to be written to the Pattern Generator. This method is an experimental feature in the current PYNQ image. It will not update the Pattern Generator waveforms which will be cleared when this method is called. The Pattern Generator needs to be set up and in the READY state before `update_patter()` can be called. The following code shows how to use the `update_pattern()`.
First, we define values for the number of samples. This is similar to what we did for the Wavedrom patterns, except this time we will generate arrays of 1's and 0's. 


In [ ]:
WS2812 = False 

if(WS2812 is True):
    T0H_PULSES = 3
    T1H_PULSES = 6
    T0L_PULSES = 5
    T1L_PULSES = 5
    TLL_PULSES = 60 # Not needed unless trying to send multiple sequential patterns quickly. 
else: # WS2813
    T0H_PULSES = 3
    T1H_PULSES = 8
    T0L_PULSES = 8 
    T1L_PULSES = 4
    
t0h = [1] * T0H_PULSES
t1h = [1] * T1H_PULSES
t0l = [0] * T0L_PULSES
t1l = [0] * T1L_PULSES

bit0 = t0h + t0l
bit1 = t1h + t1l

Next we setup the Pattern Generator, but with a dummy initial pattern.  

In [ ]:
dummy_pattern = 'l' # This won't be used. We will update the pattern in the next cell
waveform = {'signal': [
    ['led',
        {'name': 'bit0', 'pin': 'D0', 'wave': dummy_pattern}], 
    ], 
    'foot': {'tock': 1},
    'head': {'text': 'waveform'}}
    
pattern_generator.setup(waveform,
                        stimulus_group_name='led', frequency_mhz=10, initial_value=0)

In [ ]:
# Convert a list of numbers to the serial array required by the Pattern Generator
def array_to_pattern(array):
    pattern = []
    for i, _ in enumerate(array):
        for j in range(23,-1, -1): # walk through 24 bits
            bit_test = array[i] & (1<<j)
            # Append a bit0/bit1 pattern
            if bit_test is 0:
                pattern = pattern+bit0 
            else:
                pattern = pattern+bit1    
    return pattern

In [ ]:
import random

# Generate some random RGB values 
for i in range(NUMBER_OF_LEDS):
    led_array[i] = random.randint(0,0x0f0f0f) # restrict maximum value so LEDs are not too bright
led_pattern= array_to_pattern(led_array)
pattern_generator.update_pattern(led_pattern)
pattern_generator.run()

In [ ]:
pattern_generator.reset()

# Widgets

Widgets are components that you can add to a Jupyter notebooks to add interactivity. For example, you can add a textbox, a clickable button, dropdown boxes, slides and more to control behaviour of functions in your notebook. 

See a [list of widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) on the iPython documentation. 

Reference: https://ipywidgets.readthedocs.io

In [ ]:
import ipywidgets as widgets
from IPython.display import display

## Button

A button widget can be clicked, in this case "on" or "off". We will use this to turn on and off all the LEDs. 

In [ ]:
# Setup() PG
dummy_pattern = 'l' # This won't be used, we will update the pattern in the next cell
waveform = {'signal': [
    ['led',
        {'name': 'bit0', 'pin': 'D0', 'wave': dummy_pattern}], 
    ], 
    'foot': {'tock': 1},
    'head': {'text': 'waveform'}}
pattern_generator.setup(waveform,
                        stimulus_group_name='led', frequency_mhz=10, initial_value=0)

# Create the toggle button
myToggleButton = widgets.ToggleButtons(
    options=['off', 'on']
)

# This function is called with the button is clicked
def toggle_leds(button_value):
    # Check the value of the button (value.new)
    if(button_value.new is "off"):
        for i, _ in enumerate(led_array):
            led_array[i] = 0 # Turn all LEDs "off"     
        led_pattern= array_to_pattern(led_array)
        pattern_generator.update_pattern(led_pattern)
        pattern_generator.run()
    else:
        for i, _ in enumerate(led_array):
            led_array[i] = 0x707070 # Set all LEDs to the same "on" value
        led_pattern= array_to_pattern(led_array)
        pattern_generator.update_pattern(led_pattern)
        pattern_generator.run()

# The button will be "off" by default, so set all the LEDs to off initially
for i, _ in enumerate(led_array):
    led_array[i] = 0
led_pattern= array_to_pattern(led_array)
pattern_generator.update_pattern(led_pattern)
pattern_generator.run()
        
# Observe the button and call the toggle_leds function when it is clicked. 
# Pass the *value* of the button to the function (this will be "on" or "off")
myToggleButton.observe(toggle_leds,'value')

# Display the button
display(myToggleButton)

In [ ]:
pattern_generator.reset()

## color picker 

The color picker widget allows you to select a color from the color palette, or enter a value for the color. This example will show how to add the color picker widget and use it to set the same color to all the LEDs in the string. 

Note that brighter color values may look different to the color picker. 

In [ ]:
# Setup() PG
dummy_pattern = 'l' # This won't be used, we will update the pattern in the next cell
waveform = {'signal': [
    ['led',
        {'name': 'bit0', 'pin': 'D0', 'wave': dummy_pattern}], 
    ], 
    'foot': {'tock': 1},
    'head': {'text': 'waveform'}}
pattern_generator.setup(waveform,
                        stimulus_group_name='led', frequency_mhz=10, initial_value=0)

# Create the toggle button
myToggleButton = widgets.ToggleButtons(
    options=['off', 'on']
)

# Create the colorPicker object
myColorPicker = widgets.ColorPicker(
    description='Pick a color',
    value='black' # Set an initial value (black is off)
)

def write_color(color):
    # Color value is a hex string with a leading '#' character, and is Red, Green, Blue (RGB) format 
    # LEDs need Green, Red, Blue (GRB)
    
    RGB = int(color.new[1:], 16) # remove '#' and convert string to number
    
    #Split into red, green, blue components
    red = (RGB >> 16) & 0xff
    green = (RGB >> 8) & 0xff
    blue = RGB & 0xff
    
    # Convert to GRB format
    GRB = (green << 16) + (red << 8) + blue
    
    # Update array. All pixels will get the same value
    for i, _ in enumerate(led_array):
        led_array[i] = GRB
         
    led_pattern= array_to_pattern(led_array)
    pattern_generator.update_pattern(led_pattern)
    pattern_generator.run()

    
# The default value is black, or "off", so set all the LEDs to off initially
for i in (led_array):
    led_array[i] = 0           
     
led_pattern= array_to_pattern(led_array)
pattern_generator.update_pattern(led_pattern)
pattern_generator.run()


# Observe the button and call the toggle_leds function when it is clicked. 
# Pass the *value* of the colorPicker to the function (this will be a string e.g. black: #000000)
myColorPicker.observe(write_color,'value')

# Display the colorPicker
display(myColorPicker)

In [ ]:
pattern_generator.reset()

# Next steps: Build your own project

This notebook shows how to use the LogicTools *Pattern Generator* to create patterns that can be used to control external peripherals. In this design an LED controller was created. 
You also saw how to use iPython Widgets to add interactive elements to your notebook. 

You can extend this notebook to build more elaborate LED displays, or add more widgets or functionality to the functions the existing widgets control. The buttons and switches on the board can also be used to control your LED. See the Buttons and Switches example on the board in the directory: base/board/board_btns_leds.ipynb.

For example, create functions to
* Flash the LEDs
* Increase or decrease the speed the LEDs flash
* Build an LED counter
* Display some interesting patterns
* Cycle through each color value for each LED

Extend the widget examples
* Add a button to select individual LEDs, and then use the color picker to set the color for each LED. 
* Add a new dropdown box widget to select colors for LEDs. 


Build some applications
* Build a traffic light. Use an LED for the STOP, GO, and AMBER light. Use another LED for a pedestrian crossing. Use widgets to press a button for the pedestrian crossing to change the light sequence. 
* Build a 1 dimension (1D) game on your LED string. e.g. https://www.instructables.com/Make-Your-Own-1D-Pong-Game/. Can you come up with your own game?



Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----